In [1]:
!pip install torchmetrics

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [3]:
FILL = "FILL"

In [4]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-04-25 18:29:07--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: 'smsspamcollection.zip'

smsspamcollection.z     [  <=>               ] 198.65K   715KB/s    in 0.3s    

2024-04-25 18:29:08 (715 KB/s) - 'smsspamcollection.zip' saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [5]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [6]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [7]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2024-04-25 18:29:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: 'smsspamcollection.zip.1'

smsspamcollection.z     [  <=>               ] 198.65K   678KB/s    in 0.3s    

2024-04-25 18:29:09 (678 KB/s) - 'smsspamcollection.zip.1' saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [8]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
text = df.text.values
labels = df.label.values

In [10]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', do_lower_case=True)
tokenizer.pad_token = tokenizer.eos_token
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'right'

In [13]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

Sorry da thangam.it's my mistake.

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ Sorry    │       14385 │
├──────────┼─────────────┤
│ Ġda      │       12379 │
├──────────┼─────────────┤
│ Ġthan    │         621 │
├──────────┼─────────────┤
│ gam      │       28483 │
├──────────┼─────────────┤
│ .        │          13 │
├──────────┼─────────────┤
│ it       │         270 │
├──────────┼─────────────┤
│ 's       │         338 │
├──────────┼─────────────┤
│ Ġmy      │         616 │
├──────────┼─────────────┤
│ Ġmistake │        7457 │
├──────────┼─────────────┤
│ .        │          13 │
├──────────┼─────────────┤
│ Ċ        │         198 │
╘══════════╧═════════════╛


In [21]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  return tokenizer.encode_plus(input_text, add_special_tokens=True, max_length=32, pad_to_max_length=True, return_attention_mask=True,  return_tensors='pt',)

# https://stackoverflow.com/questions/61708486/whats-difference-between-tokenizer-encode-and-tokenizer-encode-plus-in-hugging

for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = torch.cat(token_id, 0)
attention_masks = torch.cat(attention_masks, 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/shumailsajjad/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [27]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['O', 'ic', 'Ġcos', 'Ġme', 'Ġn', 'Ġmy', 'Ġs', 'is', 'Ġgot', 'Ġno', 'Ġlunch', 'Ġtoday', 'Ġmy', 'Ġdad', 'Ġwent', 'Ġout', '...', 'ĠSo', 'Ġdunno', 'Ġwhether', 'Ġ2', 'Ġeat', 'Ġin', 'Ġsch', 'Ġor', 'Ġwat', '...', 'Ċ', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ O             │          46 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ ic            │         291 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġcos          │        8615 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġme           │         502 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġn            │         299 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġmy           │         616 │                1 │
├──────

In [29]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(np.arange(len(labels)), test_size=val_ratio, random_state=42, stratify=labels)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], labels[train_idx])

val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(train_set, batch_size, sampler=RandomSampler(train_set))

validation_dataloader = DataLoader(val_set, batch_size, sampler=SequentialSampler(val_set))

### Load specific versions of the model

In [31]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = GPT2Config.from_pretrained('gpt2', output_attentions=False, output_hidden_states=False)

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [32]:
# If on GPU, do as below
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # should be cpu as using jupyter notebook

In [35]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [42]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = False

# Turn off gradients using the above
if fine_tune:
    for param in model.parameters():
        param.requires_grad = True

total_parameters = 0
for n, p in model.named_parameters(): 
    total_parameters += p.numel()

assert(total_parameters == 124441344)

### Train the model

In [43]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
# spam vs not spam (binary)
accuracy = Accuracy(task='binary', num_classes=2, average='macro')
recall = Recall(task='binary', num_classes=2, average='macro')
precision = Precision(task='binary', num_classes=2, average='macro')
auroc = AUROC(task='binary', num_classes=2) 

In [44]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        ids, mask, labels = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(ids, token_type_ids=None, attention_mask=mask, labels=labels)

        # Backward pass
        train_output.loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        ids, mask, batch_labels = batch

        with torch.no_grad():
          # Forward pass
            eval_output = model(ids, token_type_ids=None, attention_mask=mask)

        # Calculate validation metrics
        labels = batch_labels
        predicted_labels = torch.argmax(eval_output.logits, 1)

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(eval_output.logits.softmax(dim=1)[:,1], labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:  50%|██████████████████▌                  | 1/2 [02:34<02:34, 154.83s/it]


	 - Train loss: 0.0237
	 - Validation Accuracy: 0.9929
	 - Validation Precision: 0.8857
	 - Validation Recall: 0.8524
	 - Validation AUROC: 0.9103



Epoch: 100%|█████████████████████████████████████| 2/2 [05:07<00:00, 153.70s/it]


	 - Train loss: 0.0113
	 - Validation Accuracy: 0.9795
	 - Validation Precision: 0.8405
	 - Validation Recall: 0.8881
	 - Validation AUROC: 0.9133



### Test on a specific sentence, see the outcome

In [41]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


/Users/shumailsajjad/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

### Results

#### 1) Fine-Tuning GPT-2 and Classifier Head
    
     - Train loss: 0.0322
	 - Validation Accuracy: 0.9920
	 - Validation Precision: 0.8786
	 - Validation Recall: 0.8619
	 - Validation AUROC: 0.9130
        
#### 2) Using GPT-2 as Feature Encoder

     - Train loss: 0.0113
	 - Validation Accuracy: 0.9795
	 - Validation Precision: 0.8405
	 - Validation Recall: 0.8881
	 - Validation AUROC: 0.9133

### Conclusion
Fine-tuning both GPT-2 and the classifier head leads to higher overall accuracy and precision, though using GPT-2 as a feature encoder offers a better recall and almost same AUROC, indicating slight effectiveness even without fine-tuning the GPT2 and the classifier head.